In [9]:
# !pip install --upgrade tensorflow --user
# !pip install keras --user
#!pip install unidecode


    100% |████████████████████████████████| 245kB 39.8MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [217]:
import io, json, talos, itertools, random
from collections import Counter

from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.layers import Input, Concatenate, concatenate
import numpy as np
import unidecode

from keras.models import Model

In [197]:
with open("./data/all_train_data.json", 'r') as f:
    data = json.load(f)

In [214]:
with open("./data/train_data_lga.json", 'r') as f:
    data_lga = json.load(f)

In [215]:
with open("./data/train_data_elf.json", 'r') as f:
    data_elf = json.load(f)

In [222]:
random.shuffle(data_elf)

In [227]:
elf_data_1 = data_elf[0:int(len(data_elf)/2)]
elf_data_2 = data_elf[int(len(data_elf)/2):]

In [228]:
data = data_lga + elf_data_1

In [229]:
MAX_WORDS = 5000
EMBEDDING_SIZE = 200
MAX_LENGTH = 200

In [230]:
text = list(itertools.chain.from_iterable([[contrib["answer"],contrib["question"]] for contrib in data]))

In [231]:
tokenizer = Tokenizer(num_words=MAX_WORDS, lower=True)
tokenizer.fit_on_texts(text)

In [232]:
def sequences_from_list_of_text(text_list):
    sequences = tokenizer.texts_to_sequences([unidecode.unidecode(text) for text in text_list])
    return pad_sequences(sequences, maxlen=MAX_LENGTH)

In [233]:
data_questions = sequences_from_list_of_text([contrib["question"] for contrib in data])
data_answers = sequences_from_list_of_text([contrib["answer"] for contrib in data])

In [289]:
TAGS_TO_REMOVE = ["Autres", "Sans réponse / Hors sujet / Inclassable"]

In [290]:
data_tags = [contrib["target"] for contrib in data if contrib]
all_tags = [elt for elt in itertools.chain.from_iterable(data_tags) if elt not in TAGS_TO_REMOVE]
all_tags_count = Counter(all_tags)
sup_100_tags = [key for key, value in dict(all_tags_count).items() if value >10]

In [291]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit([sup_100_tags])
classes = multilabel_binarizer.classes_
target = [[e for e in elt if e in multilabel_binarizer.classes_] for elt in data_tags]
y = multilabel_binarizer.transform(target)

In [292]:
x_question_train, x_question_test, \
x_answer_train, x_answer_test,\
y_train, y_test = train_test_split(data_questions, data_answers, y, test_size=0.1, random_state=42)

# Siameese 

In [293]:
question_input = Input(shape=(MAX_LENGTH,))
answer_input = Input(shape=(MAX_LENGTH,))

x_question = Embedding(MAX_WORDS, EMBEDDING_SIZE, input_length=MAX_LENGTH)(question_input)

x_question = Dropout(0.1)(x_question)
x_question = Conv1D(200, 3, padding='valid', activation='relu', strides=1)(x_question)
x_question = GlobalMaxPool1D()(x_question)
x_question = Dense(100)(x_question)

x_answers = Embedding(MAX_WORDS, EMBEDDING_SIZE, input_length=MAX_LENGTH)(answer_input)
x_answers = Dropout(0.1)(x_answers)
x_answers = Conv1D(200, 3, padding='valid', activation='relu', strides=1)(x_answers)
x_answers = GlobalMaxPool1D()(x_answers)
x_answers = Dense(100)(x_answers)

x = Concatenate(axis=1)([x_question, x_answers])
x = Dropout(0.1)(x)
x = Dense(100)(x)
x = Dropout(0.1)(x)

output = Dense(len(classes), activation="softmax")(x)

In [294]:
model = Model([question_input, answer_input],
                output)

model.compile(optimizer="adam",
               loss='binary_crossentropy', metrics=['categorical_accuracy'],
                )

model.fit([x_question_train, x_answer_train], y_train, batch_size=32,
          epochs=20,
                validation_split=0.1,
                callbacks=callbacks)

Train on 116910 samples, validate on 12990 samples
Epoch 1/20
116910/116910 [==============================] - 250s 2ms/step - loss: 0.0062 - categorical_accuracy: 0.5701 - val_loss: 0.0050 - val_categorical_accuracy: 0.6484
Epoch 2/20
116910/116910 [==============================] - 249s 2ms/step - loss: 0.0050 - categorical_accuracy: 0.6397 - val_loss: 0.0048 - val_categorical_accuracy: 0.6311
Epoch 3/20
116910/116910 [==============================] - 248s 2ms/step - loss: 0.0047 - categorical_accuracy: 0.6515 - val_loss: 0.0048 - val_categorical_accuracy: 0.6468
Epoch 4/20
116910/116910 [==============================] - 248s 2ms/step - loss: 0.0046 - categorical_accuracy: 0.6583 - val_loss: 0.0047 - val_categorical_accuracy: 0.6416
Epoch 5/20
116910/116910 [==============================] - 248s 2ms/step - loss: 0.0045 - categorical_accuracy: 0.6626 - val_loss: 0.0047 - val_categorical_accuracy: 0.6413
Epoch 6/20
116910/116910 [==============================] - 248s 2ms/step - los

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



116910/116910 [==============================] - 248s 2ms/step - loss: 0.0042 - categorical_accuracy: 0.6751 - val_loss: 0.0049 - val_categorical_accuracy: 0.6551
Epoch 9/20
 61696/116910 [==============>...............] - ETA: 1:54 - loss: 0.0041 - categorical_accuracy: 0.6780

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



116910/116910 [==============================] - 248s 2ms/step - loss: 0.0041 - categorical_accuracy: 0.6778 - val_loss: 0.0048 - val_categorical_accuracy: 0.6511
Epoch 11/20
 51424/116910 [============>.................] - ETA: 2:15 - loss: 0.0040 - categorical_accuracy: 0.6822

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [295]:
metrics = model.evaluate([x_question_test, x_answer_test], y_test)
print("{}: {}".format(model.metrics_names[0], metrics[0]))
print("{}: {}".format(model.metrics_names[1], metrics[1]))

14434/14434 [==============================] - 6s 433us/step
loss: 0.004952189853505613
categorical_accuracy: 0.6531799916862963


In [296]:
predictions = model.predict([x_question_test, x_answer_test])

In [297]:
import tensorflow as tf

In [298]:
sess = tf.Session()

def evaluation_top_k(y_test, y_pred):
    with sess.as_default():

        return {f"top_{k}":tf.keras.metrics.top_k_categorical_accuracy(y_test, y_pred,k=k    ).eval() 
                for k in [1, 2, 3, 5]}

In [299]:
evaluation_top_k(y_test, predictions)

{'top_1': 0.65318,
 'top_2': 0.78571427,
 'top_3': 0.8322017,
 'top_5': 0.87972844}

In [300]:
test_question = "Qu'est-ce qui pourrait vous inciter à changer vos comportements qui ne sont pas écologiques ? N'hésitez pas à donner des exemples concrets"
test_answer = "Réduire les déchets Covoiturage Réutilisation"

In [301]:
feature_test_question = sequences_from_list_of_text([test_question])
feature_test_answer = sequences_from_list_of_text([test_answer])

In [302]:
prediction =  model.predict([feature_test_question, feature_test_answer])[0]

In [303]:
list(sorted(zip(multilabel_binarizer.classes_, prediction), key= lambda x: x[1], reverse=True))[0:10]

[('Recherche, innovation, rentabilité', 0.23444806),
 ('Transports en commun meilleurs', 0.1592456),
 ('Aides financières, fiscalité incitative, baisse prix', 0.14431588),
 ('Politique publique forte, long terme', 0.08663409),
 ('Tri sélectif plus facile', 0.07258257),
 ('Réduire les emballages (favoriser vrac)', 0.06820785),
 ('Taxer les gros pollueurs', 0.060008597),
 ('Pistes cyclables', 0.050323106),
 ('Taxer les produits et services non écolo', 0.024411881),
 ('Transports en commun moins chers', 0.021235093)]

# Test on ELF Data

In [309]:
data_test_questions = sequences_from_list_of_text([contrib["question"] for contrib in elf_data_2])
data_test_answers = sequences_from_list_of_text([contrib["answer"] for contrib in elf_data_2])
data_test_tags = [contrib["target"] for contrib in elf_data_2]

In [322]:
len(data_test_questions)

6211

In [310]:
prediction_elf = model.predict([data_test_questions, data_test_answers])

In [311]:
data_to_send = []

for prediction_vector, data_tmp in zip(prediction_elf, to_test.to_dict(orient="records")):
    clean_prediction = list(sorted(zip(multilabel_binarizer.classes_, prediction_vector), key= lambda x: x[1], reverse=True))[0:10]
    for pred in clean_prediction:
        data_to_send.append({
            "predicted_tag":pred[0],
            "score":pred[1], 
            "tags":data_tmp["tag"], 
            "answer":data_tmp["corpus"], 
            "question":data_tmp['question_elf'], 
            "question_id":data_tmp['question'],
            "contribution":data_tmp['contribution'],
        })
        

In [312]:
df_results = pd.DataFrame(data_to_send)

## Filter on ELF tags

In [314]:
df_elf_tags = pd.read_csv("./data/elf_tags.csv", sep=";")

In [319]:
df_results.head()

,answer,contribution,predicted_tag,question,question_id,score,tags
0,Réfléchir à un vrai programme d'Education à la...,281,Arrêt lobbying,Quelles sont les discriminations les plus répa...,DC12,0.258730,{éduquer / former au vivre ensemble dès le plu...
1,Réfléchir à un vrai programme d'Education à la...,281,Mobilisation citoyenne,Quelles sont les discriminations les plus répa...,DC12,0.239269,{éduquer / former au vivre ensemble dès le plu...
2,Réfléchir à un vrai programme d'Education à la...,281,Exemplarité des politiques,Quelles sont les discriminations les plus répa...,DC12,0.182854,{éduquer / former au vivre ensemble dès le plu...
3,Réfléchir à un vrai programme d'Education à la...,281,Faire de la transition la priorité nationale,Quelles sont les discriminations les plus répa...,DC12,0.070139,{éduquer / former au vivre ensemble dès le plu...
4,Réfléchir à un vrai programme d'Education à la...,281,Favoriser le nucléaire,Quelles sont les discriminations les plus répa...,DC12,0.041222,{éduquer / former au vivre ensemble dès le plu...


In [321]:
df_results[df_results.predicted_tag.isin(df_elf_tags.tag)].to_csv("./validation_elf_tags.csv", sep=";")

In [316]:
df_elf_tags

,question_id,tag
0,DC11,instauration R.I.C.
1,DC11,meilleure information sur les enjeux et les ca...
2,DC11,vote obligatoire
3,DC11,plus d’exemplarité des dirigeants politiques
4,DC11,éducation citoyenne de la jeunesse
5,DC11,plus de proximité entre le peuple et les dirig...
6,DC11,plus de communication sur l’importance du vote
7,DC11,"instances citoyennes de décision (jury, tirage..."
8,DC11,respect des engagements des dirigeants politiques
9,DC11,respect résultats référendum


# Optimisations

In [180]:
params = {
    'OPTIMIZER': ['Nadam', 'Adam'],
    'EMBEDDING_SIZE': [20, 50, 100],
    'KERNEL_SIZE': [2, 3],
    'FILTER_SIZE': [100, 200, 300],
    "ACTIVATION_TYPE": ["sigmoid", "softmax"]
}

def build_model(x_train, y_train, x_val, y_val, params):

        model = Sequential()
        model.add(Embedding(MAX_WORDS, params["EMBEDDING_SIZE"], input_length=MAX_LENGTH))
        model.add(Dropout(0.1))
        model.add(Conv1D(params["FILTER_SIZE"], params["KERNEL_SIZE"], padding='valid', activation='relu', strides=1))
        model.add(GlobalMaxPool1D())
        model.add(Dense(len(classes)))
        model.add(Activation(params["ACTIVATION_TYPE"]))
        
        model.compile(optimizer=params["OPTIMIZER"], loss='binary_crossentropy', metrics=['categorical_accuracy'])

        callbacks = [
            ReduceLROnPlateau(),
            EarlyStopping(patience=4),
            ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
        ]
        
        out = model.fit(x_train,
                        y_train,
                        epochs=20,
                        batch_size=32,
                        validation_split=0.1,
                        callbacks=callbacks)

        return out, model


In [181]:
scan_object = talos.Scan(x, y, model=build_model, params=params, grid_downsample=0.1)



  0%|          | 0/7 [00:00<?, ?it/s]

Train on 79186 samples, validate on 8799 samples
Epoch 1/20
79186/79186 [==============================] - 23s 285us/step - loss: 0.0228 - categorical_accuracy: 0.4041 - val_loss: 0.0178 - val_categorical_accuracy: 0.5547
Epoch 2/20
79186/79186 [==============================] - 21s 270us/step - loss: 0.0168 - categorical_accuracy: 0.5531 - val_loss: 0.0160 - val_categorical_accuracy: 0.5930
Epoch 3/20
79186/79186 [==============================] - 21s 270us/step - loss: 0.0155 - categorical_accuracy: 0.5764 - val_loss: 0.0155 - val_categorical_accuracy: 0.6015
Epoch 4/20
79186/79186 [==============================] - 21s 270us/step - loss: 0.0148 - categorical_accuracy: 0.5901 - val_loss: 0.0150 - val_categorical_accuracy: 0.6152
Epoch 5/20
79186/79186 [==============================] - 21s 271us/step - loss: 0.0143 - categorical_accuracy: 0.5989 - val_loss: 0.0148 - val_categorical_accuracy: 0.6076
Epoch 6/20
79186/79186 [==============================] - 21s 271us/step - loss: 0.013


 14%|█▍        | 1/7 [04:41<28:07, 281.23s/it]

Train on 79186 samples, validate on 8799 samples
Epoch 1/20
79186/79186 [==============================] - 32s 403us/step - loss: 0.0318 - categorical_accuracy: 0.2977 - val_loss: 0.0194 - val_categorical_accuracy: 0.4664
Epoch 2/20
79186/79186 [==============================] - 32s 400us/step - loss: 0.0176 - categorical_accuracy: 0.4941 - val_loss: 0.0170 - val_categorical_accuracy: 0.5249
Epoch 3/20
79186/79186 [==============================] - 32s 399us/step - loss: 0.0155 - categorical_accuracy: 0.5361 - val_loss: 0.0154 - val_categorical_accuracy: 0.5432
Epoch 4/20
79186/79186 [==============================] - 32s 400us/step - loss: 0.0143 - categorical_accuracy: 0.5614 - val_loss: 0.0149 - val_categorical_accuracy: 0.5765
Epoch 5/20
79186/79186 [==============================] - 32s 400us/step - loss: 0.0136 - categorical_accuracy: 0.5758 - val_loss: 0.0157 - val_categorical_accuracy: 0.5990
Epoch 6/20
79186/79186 [==============================] - 32s 400us/step - loss: 0.012


 29%|██▊       | 2/7 [10:32<25:10, 302.13s/it]

...


 71%|███████▏  | 5/7 [25:16<10:00, 300.01s/it]

Train on 79186 samples, validate on 8799 samples
Epoch 1/20
79186/79186 [==============================] - 80s 1ms/step - loss: 0.0239 - categorical_accuracy: 0.4183 - val_loss: 0.0185 - val_categorical_accuracy: 0.5467
Epoch 2/20
79186/79186 [==============================] - 94s 1ms/step - loss: 0.0154 - categorical_accuracy: 0.5497 - val_loss: 0.0180 - val_categorical_accuracy: 0.5859
Epoch 3/20
79186/79186 [==============================] - 94s 1ms/step - loss: 0.0140 - categorical_accuracy: 0.5768 - val_loss: 0.0175 - val_categorical_accuracy: 0.6015
Epoch 4/20
79186/79186 [==============================] - 92s 1ms/step - loss: 0.0130 - categorical_accuracy: 0.5917 - val_loss: 0.0193 - val_categorical_accuracy: 0.6035
Epoch 5/20
79186/79186 [==============================] - 94s 1ms/step - loss: 0.0124 - categorical_accuracy: 0.6033 - val_loss: 0.0202 - val_categorical_accuracy: 0.6312
Epoch 6/20
79186/79186 [==============================] - 93s 1ms/step - loss: 0.0119 - categori


 86%|████████▌ | 6/7 [35:57<06:42, 402.29s/it]

Train on 79186 samples, validate on 8799 samples
Epoch 1/20
79186/79186 [==============================] - 92s 1ms/step - loss: 0.0295 - categorical_accuracy: 0.3430 - val_loss: 0.0187 - val_categorical_accuracy: 0.5110
Epoch 2/20
79186/79186 [==============================] - 93s 1ms/step - loss: 0.0160 - categorical_accuracy: 0.5380 - val_loss: 0.0156 - val_categorical_accuracy: 0.5746
Epoch 3/20
79186/79186 [==============================] - 93s 1ms/step - loss: 0.0141 - categorical_accuracy: 0.5683 - val_loss: 0.0146 - val_categorical_accuracy: 0.6020
Epoch 4/20
79186/79186 [==============================] - 92s 1ms/step - loss: 0.0132 - categorical_accuracy: 0.5842 - val_loss: 0.0151 - val_categorical_accuracy: 0.5860
Epoch 5/20
79186/79186 [==============================] - 93s 1ms/step - loss: 0.0125 - categorical_accuracy: 0.5954 - val_loss: 0.0156 - val_categorical_accuracy: 0.5968
Epoch 6/20
79186/79186 [==============================] - 98s 1ms/step - loss: 0.0120 - categori


100%|██████████| 7/7 [46:50<00:00, 477.69s/it]

In [186]:
scan_object.details

complete_time            03/11/19/16:23
experiment_name           031119153617_
grid_downsample                     0.1
random_method          uniform_mersenne
reduce_loss                       False
reduction_interval                   50
reduction_method                   None
reduction_metric                val_acc
reduction_threshold                 0.2
reduction_window                     20
x_shape                   (125693, 200)
y_shape                   (125693, 307)
dtype: object

In [188]:
scan_object.best_model(metric="val_categorical_accuracy")

In [195]:
features = get_features("Il suffit de cesser la destruction des services publics de proximité pour des raisons financières. De plus, ces fermetures sont imposées par l'UE, qui oblige les états a privatiser tous les services publics( poste, hôpitaux, énergie...) , accentuant le désir de rentabilité des services publics actuels.")

In [190]:
best_model = scan_object.best_model(metric="val_categorical_accuracy")

In [193]:
def get_prediction_tags(features):
    return sorted(zip(classes , best_model.predict(features)[0]), key=lambda x: x[1], reverse=True)[0:10]

In [194]:
get_prediction_tags(features)

[('Z_Autres', 0.23051563),
 ('Sans_réponse_/_Hors_sujet_/_Inclassable', 0.21212253),
 ('A_Aucune', 0.10980064),
 ('A_Aucun,_ou_la_réduire', 0.0658187),
 ('A_Aucun', 0.058471948),
 ('Communes,_intercommunalités', 0.039815485),
 ('B_>_Les_4,_tout_est_lié', 0.036506474),
 ('Collectivités_locales', 0.03515768),
 ('Simple_décompte_informatif_(actuel)', 0.027210295),
 ('Compter_comme_exprimé', 0.026890397)]

In [192]:
best_model.predict(features)[0]

array([2.00256705e-03, 1.95088983e-03, 2.37703323e-04, 5.95143437e-03,
       5.84719479e-02, 6.58186972e-02, 7.00008869e-03, 1.09800637e-01,
       1.61245465e-03, 4.67148423e-03, 1.10518932e-03, 1.48117542e-04,
       9.97066498e-04, 8.65161419e-05, 7.77244568e-05, 4.30643559e-05,
       3.33845615e-04, 1.47101283e-03, 8.64326954e-04, 1.01268291e-04,
       1.37299299e-04, 7.73876905e-04, 3.45498323e-04, 1.95443630e-04,
       2.46730447e-03, 3.74785066e-03, 7.32392073e-04, 1.40041113e-04,
       2.44408846e-04, 2.37083435e-03, 1.33395195e-04, 1.41367316e-03,
       6.81817532e-04, 3.53753567e-05, 4.47630882e-05, 3.65064740e-02,
       1.26600266e-04, 2.98857689e-03, 1.72555447e-05, 4.55945730e-04,
       4.31835651e-05, 8.83936882e-03, 2.69711018e-05, 1.44715309e-02,
       3.61502171e-05, 6.91711903e-05, 1.79138780e-03, 3.27557325e-04,
       3.61427665e-03, 7.78764486e-04, 3.30448151e-04, 3.51576805e-02,
       3.98154855e-02, 2.68903971e-02, 2.17854977e-05, 7.42167234e-03,
      